we will cover - 

- instruction finetuning of llms
- dataset prep for supervised instruction finetuning
- finetuning the llm
- extracting and evaluating llm-generated instruction responses

![ft4](images/ft-4.png)

<h4>Stage 1</h4>

1. 2 different prompt styles to format the dataset into - Alpaca and Phi3. Alpaca was one of the first publicly detailed instruction fine tuning and more famous, later came Microsoft's Phi3. We use Alpaca. Also divide dataset into training, testing and validation.

![ft5](images/ft-5.png)

2. Training Batches - writing custom collate functions pads the training examples in each batch to the same length while allowing different batches to hae different lengths -> minimizes unnesessary padding by only extending sequences to match the longest one in each batch, not the whole dataset.

We replace all the end-of-text token with a special token (-100 here) to exclude these padding tokens from contributing to the training loss calculation, we replace all except the first end-of-text token to let the model know when to stop though. Using -100 as the default setting of cross entropy function in pytorch is - ignore_index=-100; ignoring targets labelled with -100.

It is also common to mask out instructions in the targed tokens when calculating loss so that the model does not memorize instructions - reducing overfitting. sometimes its done, sometimes not, we are not going to currently.

![ft6](images/ft-6.png)
![ft7](images/ft-7.png)


